# 1. Clone Repository and Install Dependencies

First, we'll clone the repository from GitHub and install all required packages:

In [ ]:
!git clone https://github.com/lytuan123/MoneyPrinterTurbo.git
%cd MoneyPrinterTurbo
!pip install -q -r requirements.txt


In [ ]:
!pip install pyngrok --quiet
!apt-get install imagemagick
!apt-get install ffmpeg

!cp config.example.toml config.toml

!apt-get update && apt-get install -y ffmpeg

# 2. Configure ngrok for Remote Access

We'll use ngrok to create a secure tunnel to expose our local Streamlit server to the internet.

**Important**: You need to get your authentication token from the [ngrok dashboard](https://dashboard.ngrok.com/get-started/your-authtoken) to use this service.

### 👇 Replace YOUR_AUTHTOKEN_HERE with your ngrok authtoken

In [ ]:
# 👇 Replace YOUR_AUTHTOKEN_HERE with your ngrok authtoken
NGROK_AUTH_TOKEN = "YOUR_AUTHTOKEN_HERE"

# 3. Launch Application and Generate Public URL

Now we'll start the Streamlit server and create an ngrok tunnel to make it accessible online:

In [ ]:
import torch
import os
import time
from pyngrok import ngrok, conf

torch.cuda.empty_cache()

# Ensure no tunnels are open
ngrok.kill()

# Set the environment variable for ngrok
conf.get_default().auth_token = NGROK_AUTH_TOKEN

# Check if the Streamlit file exists
streamlit_file = "./webui/Main.py"
if not os.path.isfile(streamlit_file):
    raise FileNotFoundError(f"❌ File not found: {streamlit_file}")

# Set the current path for PYTHONPATH
os.environ["PYTHONPATH"] = os.getcwd()

# Run Streamlit in the background and redirect logs to a file for review
print("🚀 Running Streamlit...")
!streamlit run {streamlit_file} --server.enableCORS=True --browser.gatherUsageStats=False > log.txt 2>&1 &

# Wait for Streamlit to initialize
time.sleep(8)

# Check the logs for errors (optional, can be skipped)
with open("log.txt", "r") as f:
    logs = f.read()
    if "Traceback" in logs:
        print("⚠️ Error occurred during Streamlit execution:")
        print(logs.splitlines()[-20:])  # Print last 20 lines
    else:
        print("✅ Streamlit started successfully.")

# Open an ngrok tunnel
print("🌐 Creating ngrok tunnel...")
public_url = ngrok.connect(8501)
print(f"\n🔗 Access WebUI at: {public_url}")